In [1]:
#!pip install pandas numpy loompy pyscenic
!pip install "numpy<1.24" --force-reinstall
import loompy
import pyscenic
import subprocess
import pandas as pd
import numpy as np
import glob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 33.9 MB/s  0:00:00 36.2 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyscenic 0.12.1 requires multiprocessing-on-dill, which is not installed.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.1.1 which is incompatible.


In [6]:
# File loom
f_loom_path_scenic = "TimeCourse.loom"
ds = loompy.connect(f_loom_path_scenic, mode='r')

loom_file = "TimeCourse.loom"
ds = loompy.connect(loom_file, mode='r')

# Voir les dimensions
print("Dimensions :", ds.shape)  # (nb_genes, nb_cellules)

# Voir les noms de lignes et colonnes
print("Exemples de noms de gènes :", ds.ra.keys())
print("Exemples de noms de cellules :", ds.ca.keys())

# Aperçu
print("Premier gène :", ds.ra["Gene"][0])
print("Première cellule :", ds.ca["CellID"][0])

# Liste des attributs disponibles
print("\nAttributs ligne (row attributes) :", list(ds.ra.keys()))
print("Attributs colonne (column attributes) :", list(ds.ca.keys()))
print("Attributs globaux :", list(ds.attrs.keys()))

print(ds[:, :5])
print(np.mean(ds[:, :5])) 
ds.close()




Dimensions : (18723, 20671)
Exemples de noms de gènes : ['Gene']
Exemples de noms de cellules : ['ADT.weight', 'CellID', 'RNA.weight', 'RNA_snn_res.0.15', 'SingleR', 'doublet_finder', 'final_annot', 'manual_annotation', 'mapping.score', 'nCount_ADT', 'nCount_RNA', 'nFeature_ADT', 'nFeature_RNA', 'orig.ident', 'percent.mt', 'predicted.celltype.l1', 'predicted.celltype.l1.score', 'predicted.celltype.l2', 'predicted.celltype.l2.score', 'predicted.celltype.l3', 'predicted.celltype.l3.score', 'qc_pass', 'seurat_clusters', 'timepoint', 'wsnn_res.0.15']
Premier gène : OR4F5
Première cellule : D1_AAACCCAAGACCAGCA-1

Attributs ligne (row attributes) : ['Gene']
Attributs colonne (column attributes) : ['ADT.weight', 'CellID', 'RNA.weight', 'RNA_snn_res.0.15', 'SingleR', 'doublet_finder', 'final_annot', 'manual_annotation', 'mapping.score', 'nCount_ADT', 'nCount_RNA', 'nFeature_ADT', 'nFeature_RNA', 'orig.ident', 'percent.mt', 'predicted.celltype.l1', 'predicted.celltype.l1.score', 'predicted.cell

In [9]:

ds = loompy.connect("TimeCourse.loom", mode='r')
print("Dimensions :", ds.shape)
print("Attributs ligne :", list(ds.ra.keys()))
print("Attributs colonne :", list(ds.ca.keys()))
print("Premier gène :", ds.ra['Gene'][0])
print("Première cellule :", ds.ca['CellID'][0])
print("Valeurs d'expression (extrait) :", ds[:5, :5])
ds.close()


Dimensions : (18723, 20671)
Attributs ligne : ['Gene']
Attributs colonne : ['ADT.weight', 'CellID', 'RNA.weight', 'RNA_snn_res.0.15', 'SingleR', 'doublet_finder', 'final_annot', 'manual_annotation', 'mapping.score', 'nCount_ADT', 'nCount_RNA', 'nFeature_ADT', 'nFeature_RNA', 'orig.ident', 'percent.mt', 'predicted.celltype.l1', 'predicted.celltype.l1.score', 'predicted.celltype.l2', 'predicted.celltype.l2.score', 'predicted.celltype.l3', 'predicted.celltype.l3.score', 'qc_pass', 'seurat_clusters', 'timepoint', 'wsnn_res.0.15']
Premier gène : OR4F5
Première cellule : D1_AAACCCAAGACCAGCA-1
Valeurs d'expression (extrait) : [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [3]:
#STEP 1: Gene regulatory network inference, and generation of co-expression modules¶
#transcription factors list
f_tfs = "allTFs_hg38.txt" # human

In [4]:
!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 20

/home/a.blanc-boekholt/miniconda3/envs/scenic_env/lib/python3.9/site-packages/ctxcore/__init__.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution

2025-10-24 12:04:13,186 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-10-24 12:04:20,259 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in

In [ ]:
adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')
adjacencies.head()

In [ ]:
#STEP 2-3: Regulon prediction aka cisTarget from CLI

# ranking databases
f_db_glob = "hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.genes_vs_motifs.rankings.feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl"

In [ ]:
!pyscenic ctx adj.tsv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20

In [ ]:
#STEP 4: Cellular enrichment (aka AUCell) from CLI
nGenesDetectedPerCell = np.sum(adata.X>0, axis=1)
percentiles = nGenesDetectedPerCell.quantile([.01, .05, .10, .50, 1])
print(percentiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=150)
sns.distplot(nGenesDetectedPerCell, norm_hist=False, kde=False, bins='fd')
for i,x in enumerate(percentiles):
    fig.gca().axvline(x=x, ymin=0,ymax=1, color='red')
    ax.text(x=x, y=ax.get_ylim()[1], s=f'{int(x)} ({percentiles.index.values[i]*100}%)', color='red', rotation=30, size='x-small',rotation_mode='anchor' )
ax.set_xlabel('# of genes')
ax.set_ylabel('# of cells')
fig.tight_layout()

In [ ]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 20

In [ ]:
# Visualization of SCENIC's AUC matrix
import json
import zlib
import base64

# collect SCENIC AUCell output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [ ]:
import umap

# UMAP
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric='correlation').fit_transform
dr_umap = runUmap( auc_mtx )
pd.DataFrame(dr_umap, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_umap.txt", sep='\t')
# tSNE
tsne = TSNE( n_jobs=20 )
dr_tsne = tsne.fit_transform( auc_mtx )
pd.DataFrame(dr_tsne, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_tsne.txt", sep='\t')